As **árvores de decisão** são modelos de aprendizado de máquina supervisionados, o que significa que são criadas a partir de um conjunto de treinamento de dados rotulados. A criação da árvore é onde o aprendizado de máquina acontece.

O funcionamento da Árvore de Decisão visa formar “caminhos” que vão dividindo os dados em pequenos grupos. Essa divisão acontece com base nas características dos dados para que, no final, possamos entrar com um novo registro e o modelo nos dizer em qual classe esse dado se encaixa melhor.

Eles também são a base do funcionamento de outros poderosos algoritmos, como o Random Forest, por exemplo.

### Exemplo 1

In [ ]:
from IPython.display import Image
Image('tabela 1.png')

In [ ]:
Image('árvore de decisão 1.1.png')

É importante observamos que esta mesma tabela poderia levar a construção de uma árvore diferente.

In [ ]:
Image('árvore de decisão 1.2.png')

Neste simples exemplo vemos que normalmente não existirá uma única árvore de decisão para um mesmo problema, sendo que com diferentes árvores poderemos chegar a um mesmo resultado.

## Definindo os nós e ramos

Assim como podemos ter mais de uma árvore para um mesmo problema, também podemos utilizar diferentes métodos de cálculo na criação de uma árvore de decisão.

Estes métodos são os responsáveis pela definição da estrutura e resultado final da árvore, e tentam buscar a estrutura mais otimizada para o problema em questão.

### Entropia

Através da entropia o algoritmo verifica como os dados estão distribuídos nas variáveis preditoras de acordo com a variação da variável target. **Quanto maior a entropia, maior a desordem dos dados; e quanto menor, maior será a ordem destes dados**, quando analisados pela ótica da variável target.

In [ ]:
Image('Entropia.png')

Nosso objetivo é construir uma árvore que, partindo do conjunto de dados inteiro, criar ramificações baseadas em condições que minimizem a entropia e aumentem o ganho de informação.

In [ ]:
Image('equação entropia.png')

### Ganho de Informação

O ganho de informação é uma medida que nos diz o quão bem uma variável preditora do conjunto de dados separa os registros conforme as suas classes.

In [ ]:
Image('equação ganho de informação.png')

O número de filhos presentes no somatório depende de como a ramificação será feita, mas normalmente são gerados dois nós filhos. Onde o peso para cada um desses filhos é calculado dividindo o total de elementos em um nó filho dividido pelo número de elementos no nó pai:

In [ ]:
Image('equação peso filho.png')

In [ ]:
Image('ganho de informação.png')

### Exemplo 2

In [ ]:
Image('tabela 2.png')

In [ ]:
Image('árvore de decisão 2.png')

#### Passo 1. Calcular a entropia da variável target

In [ ]:
Image('Entropia variável alvo.png')

#### Passo 2. Cálculo da Entropia e ganho de informação de cada variável preditora/recurso

Entropia Histórico de Crédito

In [ ]:
Image('Entropia histórico de crédito 1.1.png')

In [ ]:
Image('Entropia histórico de crédito 1.2.png')

In [ ]:
Image('Entropia histórico de crédito 2.png')

Entropia Dívida

In [ ]:
Image('Entropia Dívida 1.png')

**Exercício**

Calcule a entropia e o ganho de informação da variável `renda anual`

In [ ]:
Image('Entropia renda 1.png')

In [ ]:
Image('Entropia renda 2.png')

### Ganho de informação de cada variável

- Histórico de Crédito = 0,26
- Dívida = 0,06
- Garantias = 0,20
- Renda = 0,66

In [ ]:
Image('ramos renda.png')

### Exemplo 3 - Python/Sklearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
alunos = pd.read_csv('dados_alunos.csv')
x = alunos[['hours_studied','practice_test']]
y = alunos[['passed_exam']]

In [ ]:
#Dividindo o conjunto entre treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=0, test_size=0.2)

In [ ]:
# Criando a Árvore de Decisão
arvore_decisao = DecisionTreeClassifier(criterion = 'entropy')

In [ ]:
# Treinando o modelo com os dados do conjunto de treinamento e imprimindo a profundidade da árvore
arvore_decisao.fit(x_train, y_train)
print(f'Profundidade da Árvore de Decisão: {arvore_decisao.get_depth()}')

In [ ]:
# Classificando com o conjunto de teste e imprimindo a acurácia do modelo no conjunto de teste
y_pred = arvore_decisao.predict(x_test)
print(f'Acurácia conjunto de teste: {arvore_decisao.score(x_test, y_test)}')

In [ ]:
# Visualizando a árvore
plt.figure(figsize=(27,12))
tree.plot_tree(arvore_decisao,feature_names = x_train.columns,class_names = ['não passou', 'passou'], label='all' ,filled=True)
plt.tight_layout()
plt.show()

## Vantagens e Desvantagens

### Vantagens

- Fácil interpretação
- Não precisa de normalização ou padronização
- Rápido para classificar novos registros

### Desvantagens

- Geração de árvores muito complexas
- Pequenas mudanças nos dados podem mudar a árvore
- Problema NP-completo para contruir a árvore

# Fim